In [44]:
pip install langchain langchain_community pypdf

python(29450) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Note: you may need to restart the kernel to use updated packages.


In [45]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [56]:
pdf_path = "/Users/macbookair/Desktop/DPSI/Annuaires Statistiques METFPA/Annuaire 2024/Annuaire Stat 2023-2024_v14_21042025.pdf"

#Charger le document pdf
loader = PyPDFLoader(pdf_path)

#Découper le pdf
pages = loader.load_and_split()

print(pages[23].page_content[:100])

Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)
Ignoring wrong pointing object 67 0 (offset 0)
Ignoring wrong pointing object 77 0 (offset 0)
Ignoring wrong pointing object 80 0 (offset 0)
Ignoring wrong pointing object 82 0 (offset 0)
Ignoring wrong pointing object 87 0 (offset 0)
Ignoring wrong pointing object 89 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 99 0 (offset 0)
Ignoring wrong pointing object 111 0 (offset 0)
Ignoring wrong pointing object 126 0 (offset 0)
Ignoring wr

23 
 Tableaux du chapitre 2 : APPRENANTS  1. Répartition des Apprenants selon le Genre et la Région 


In [57]:
embeddings = OllamaEmbeddings(model="bge-m3")

vdb = Chroma.from_documents(pages, embeddings)

In [48]:
vdb._collection.count()

163

In [ ]:
template = """
Réponds dans la langue dans laquelle la question est posée, en t'appuyant uniquement
sur le contexte fourni sans ajouter d'éléments. Si la réponse n'est pas disponible
Indique : Cette information ne se trouve pas encore dans les documents à ma connaissance.
Reponds surtout de manière courtoise et agréble en ajoutant des émoticones.

Contexte : {context}
Question : {question} [/INST] 
""" 

QA_PROMPT = PromptTemplate.from_template(template)

In [54]:
llm = ChatOllama(model="mistral:7b-instruct", temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vdb.as_retriever(),
    chain_type_kwargs = {"prompt": QA_PROMPT}
)

In [ ]:
question = "Quel est le nombre d'apprenant fille en couture ?"
response = qa_chain.run(question)
response

" Il n'y a pas de nombre spécifique indiqué pour les filles en couture dans les données fournies. Les données montrent le nombre total d'employés par région, mais il n'y a pas de distinction entre les sexes pour chaque profession."